# Tool Agent

In [1]:
!pip install python-dotenv langchain-community wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
model_path = os.getenv("MODEL_PATH")

In [3]:
import logging

from app.llm.llamacpp.service import LlamaCppService

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info(f"Model path: {model_path}")
llm = LlamaCppService(model_path=model_path, n_gpu_layers=-1)

INFO:__main__:Model path: D:\llm-models\NousResearch\Hermes-2-Pro-Llama-3-8B-GGUF\Hermes-2-Pro-Llama-3-8B-Q8_0.gguf
llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from D:\llm-models\NousResearch\Hermes-2-Pro-Llama-3-8B-GGUF\Hermes-2-Pro-Llama-3-8B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32        

In [4]:
from app.tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun
from app.agents.react.agent import ReActAgentFactory

tools = [LangchainWikipediaQueryRun(top_k_results=3, doc_content_chars_max=2048)]
react_agent = ReActAgentFactory.start(llm=llm, tools=tools, step_limit=6)

In [5]:
import json
from app.core.messages import Query

future = react_agent.ask(Query(goal="What year did the iphone come out?"))
response = future.get(timeout=120)
print(f"Steps: {json.dumps([step.model_dump() for step in response.metadata['steps']], indent=2)}")
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

future = react_agent.ask(Query(goal="What other models exist?"))
response = future.get(timeout=120)
print(f"Steps: {json.dumps([step.model_dump() for step in response.metadata['steps']], indent=2)}")
print(f"Total tokens: {sum(step.token_usage.total_tokens for step in response.metadata['steps'])}")
print(response.final_output)

react_agent.stop()

Steps: [
  {
    "state_name": "States.THOUGHT",
    "next_state": "States.ACTION",
    "prompt": "You're a helpful assistant. You solve problems by breaking them down into multiple steps, thinking on those steps, acting on them, and observing. Given the problem, you will use your tools to solve it in as few steps as possible.\n\nCreate a thought based on the message.\n\nThought instructions:\n- Provides exact details on the task to best answer the message, do not forget important details.\n- You have access to tools that can help you answer the message. Always try to use a tool.\n-- In your thought, recommend a tool that will help.\n-- If the problem can be solved using the tool, always use the tool.\n-- If the problem cannot be solved or if you are unsure, give your answer with why.\n- If you keep running into issues with your tools, give your answer with the problems you're running into.\n- If the message doesn't require any tool, just give your answer.\n\nMessage:\n\"\"\"\nWhat yea

True